# Hackaton Desafio Budokai

## Equipe 11

### notebook_v5: A ideia principal nesta versão é criar um pipeline para testar vários modelos e descobrir suas perfomances.

## Bibliotecas:

In [1]:
#importanto bibliotecas

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model, preprocessing

#import statsmodels.api as sm
#from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import StandardScaler
from datetime import datetime

from sklearn.linear_model import Lasso, LassoCV
from sklearn.impute import SimpleImputer

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split

#### Lendo csv e removendo algumas colunas redundantes:

In [2]:
Y_train=pd.read_csv('shared/bases_budokai_ufpr/produtividade_soja_modelagem.csv')
df=pd.read_csv('shared/bases_budokai_ufpr/agroclimatology_budokai.csv')

In [3]:
Y_train.head()

,nivel,codigo_ibge,name,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,MU,4100103,Abatiá,2600.0,1750.0,2100.0,2600.0,2600.0,2080.0,2590.0,3350.0,2320.0,3100.0,1460.0,2640.0,2760.0,3480.0
1,MU,4100459,Altamira do Paraná,3100.0,2460.0,2950.0,2950.0,2749.0,2749.0,3250.0,3200.0,2197.0,3223.0,3471.0,3099.0,2871.0,3818.0
2,MU,4100608,Alto Paraná,1487.0,1365.0,1734.0,2727.0,2400.0,1620.0,2899.0,2899.0,1737.0,2880.0,2000.0,1800.0,2221.0,2980.0
3,MU,4100707,Alto Piquiri,2300.0,2300.0,2700.0,3193.0,2701.0,2097.0,3130.0,3310.0,2000.0,3000.0,3192.0,3488.0,2589.0,3700.0
4,MU,4100509,Altônia,1050.0,1800.0,1255.0,2355.0,2600.0,1300.0,3000.0,2773.0,815.0,2137.0,1800.0,2455.0,2780.0,2900.0


In [4]:
Y_train['nivel'].unique()

array(['MU'], dtype=object)

In [5]:
#Y_codigo=Y_train['codigo_ibge']

Y_train=Y_train.drop(columns=['nivel', 'name'])

In [6]:
Y_train.head()

,codigo_ibge,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,4100103,2600.0,1750.0,2100.0,2600.0,2600.0,2080.0,2590.0,3350.0,2320.0,3100.0,1460.0,2640.0,2760.0,3480.0
1,4100459,3100.0,2460.0,2950.0,2950.0,2749.0,2749.0,3250.0,3200.0,2197.0,3223.0,3471.0,3099.0,2871.0,3818.0
2,4100608,1487.0,1365.0,1734.0,2727.0,2400.0,1620.0,2899.0,2899.0,1737.0,2880.0,2000.0,1800.0,2221.0,2980.0
3,4100707,2300.0,2300.0,2700.0,3193.0,2701.0,2097.0,3130.0,3310.0,2000.0,3000.0,3192.0,3488.0,2589.0,3700.0
4,4100509,1050.0,1800.0,1255.0,2355.0,2600.0,1300.0,3000.0,2773.0,815.0,2137.0,1800.0,2455.0,2780.0,2900.0


### De acordo com as respostas esperadas temos:

In [7]:
codigo_saida=[4102000,4104303,4104428,4104808,4104907,4109401,4113205,4113700,4113734,4114005,4114401,4117701,4117909,4119608,4119905,4127007,4127403,4127502,4127700,4128005
]

In [8]:
dataset=pd.DataFrame(columns=df.keys())
#dataset.loc[dataset['codigo_ibge']==codigo_saida[j]]

for k in codigo_saida:
    dataset=pd.concat([df.loc[df['codigo_ibge']==k],dataset])

dataset=dataset.reset_index(drop=True)

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131500 entries, 0 to 131499
Data columns (total 38 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   data                 131500 non-null  object 
 1   codigo_ibge          131500 non-null  object 
 2   latitude             131500 non-null  float64
 3   longitude            131500 non-null  float64
 4   T2M_RANGE            131500 non-null  float64
 5   TS                   131500 non-null  float64
 6   T2MDEW               131500 non-null  float64
 7   T2MWET               131500 non-null  float64
 8   T2M_MAX              131500 non-null  float64
 9   T2M_MIN              131500 non-null  float64
 10  T2M                  131500 non-null  float64
 11  QV2M                 131500 non-null  float64
 12  RH2M                 131500 non-null  float64
 13  PRECTOTCORR          131500 non-null  float64
 14  PS                   131500 non-null  float64
 15  WS2M             

#### Transformando o codigo em inteiro novamente.

In [10]:
dataset['codigo_ibge']=list(map(int,dataset['codigo_ibge']))

#df.loc[df['codigo_ibge']==codigo_saida[0]]

### Vamos começar transformando os dados de data em 'datetime' para melhor agrupar em semanas, meses ou anos.

In [11]:
def t_data_day(data):
    data=str(data)
    data=datetime.strptime(data,"%Y%m%d")
    return data.day

def t_data_month(data):
    data=str(data)
    data=datetime.strptime(data,"%Y%m%d")
    return data.month

def t_data_year(data):
    data=str(data)
    data=datetime.strptime(data,"%Y%m%d")
    return data.year
    #return data.day, data.month, data.year

In [12]:
dataset['day']=list(map(t_data_day, dataset['data']))
dataset['month']=list(map(t_data_month, dataset['data']))
dataset['year']=list(map(t_data_year, dataset['data']))

In [13]:
dataset.head()

,data,codigo_ibge,latitude,longitude,T2M_RANGE,TS,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,CLRSKY_SFC_SW_DWN,TOA_SW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,day,month,year
0,20030101,4128005,-24.5393,-52.9865,7.99,26.27,23.23,24.83,30.21,22.21,...,30.59,42.56,133.74,165.12,18.27,0.56,2.95,1,1,2003
1,20030102,4128005,-24.5393,-52.9865,6.47,25.70,23.36,24.59,29.33,22.86,...,29.79,42.55,111.66,161.13,15.38,0.48,2.55,2,1,2003
2,20030103,4128005,-24.5393,-52.9865,6.27,24.75,22.78,23.78,28.03,21.76,...,29.66,42.53,89.88,161.48,12.85,0.41,2.12,3,1,2003
3,20030104,4128005,-24.5393,-52.9865,6.38,24.50,22.21,23.29,27.82,21.44,...,28.35,42.51,87.03,153.10,12.10,0.40,2.05,4,1,2003
4,20030105,4128005,-24.5393,-52.9865,8.43,25.62,22.50,23.95,29.68,21.24,...,29.04,42.49,52.05,156.25,7.56,0.24,1.24,5,1,2003


In [14]:
dataset=dataset.drop(dataset.loc[dataset['year']==2003].index).reset_index(drop=True)

### Podemo implementar um sistema para previsões baseado na média dos valores mensais.

In [15]:
dataset=dataset.drop(columns=['data','latitude','longitude','day'])

#### Para cada codigo postal iremos fazer um subgrupo para cada ano e agrupar os meses pela média.

#### Depois concatenamos e criamos o novo dataset.

In [16]:
codigos=dataset['codigo_ibge'].unique()
anos=dataset['year'].unique()

datanew=pd.DataFrame(columns=dataset.keys())

for i in codigos:
    for j in anos:
        aux = dataset.loc[dataset['codigo_ibge']==i].loc[dataset['year']==j].groupby(by='month').agg('mean').reset_index()
        datanew=pd.concat([datanew,aux])
    
datanew=datanew.reset_index(drop=True)

In [17]:
datanew.head()

,codigo_ibge,T2M_RANGE,TS,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M,QV2M,RH2M,...,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,TOA_SW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,month,year
0,4128005,12.313226,27.513548,19.130968,22.804839,32.921290,20.608387,26.477097,14.547742,67.569032,...,23.842258,29.756452,42.040000,129.152258,157.907419,17.224516,0.526452,-29.573548,1,2004
1,4128005,12.955517,27.584483,18.363448,22.427931,33.331379,20.375172,26.492414,13.849310,64.810690,...,23.348966,28.156207,39.736207,126.227931,149.815517,16.812759,0.507586,2.653793,2,2004
2,4128005,13.622258,26.335806,17.173871,21.365806,32.728065,19.107419,25.557097,12.963548,64.225806,...,21.451613,25.462581,35.433226,113.812581,134.426774,15.015484,0.436452,-30.048387,3,2004
3,4128005,10.766667,24.591000,19.021667,21.618333,30.242667,19.473667,24.218667,14.357000,75.240000,...,16.357333,20.808667,29.943000,88.034000,111.137667,11.629667,0.323000,-31.676667,4,2004
4,4128005,9.320645,17.494516,14.863548,16.181613,22.614839,13.293871,17.502581,11.179355,85.648387,...,11.521935,17.273548,25.104194,60.481613,90.170645,7.898387,0.194516,-31.256452,5,2004


### Precisamos colocar os dados dos meses em uma unica linha para alimentar a entrada do modelo.

#### Criando a lista de indices:

In [18]:
col=list(datanew.keys())
col.remove('year')
col.remove('codigo_ibge')
#removendo os dados que não serão escalados

In [19]:
lista_indices=[]

lista_indices.append('codigo_ibge')

for i in range(12):

    
    for j in col:
        
        aux=j+'_'+str(i+1)
        lista_indices.append(aux)
        
lista_indices.append('month')
lista_indices.append('year')

In [20]:
lista_indices

['codigo_ibge',
 'T2M_RANGE_1',
 'TS_1',
 'T2MDEW_1',
 'T2MWET_1',
 'T2M_MAX_1',
 'T2M_MIN_1',
 'T2M_1',
 'QV2M_1',
 'RH2M_1',
 'PRECTOTCORR_1',
 'PS_1',
 'WS2M_1',
 'WS2M_MAX_1',
 'WS2M_MIN_1',
 'WS2M_RANGE_1',
 'WS10M_1',
 'WS10M_MAX_1',
 'WS10M_MIN_1',
 'WS10M_RANGE_1',
 'WS50M_1',
 'WS50M_MAX_1',
 'WS50M_MIN_1',
 'WS50M_RANGE_1',
 'GWETTOP_1',
 'GWETROOT_1',
 'GWETPROF_1',
 'ALLSKY_SFC_SW_DWN_1',
 'CLRSKY_SFC_SW_DWN_1',
 'TOA_SW_DWN_1',
 'ALLSKY_SFC_PAR_TOT_1',
 'CLRSKY_SFC_PAR_TOT_1',
 'ALLSKY_SFC_UVA_1',
 'ALLSKY_SFC_UVB_1',
 'ALLSKY_SFC_UV_INDEX_1',
 'month_1',
 'T2M_RANGE_2',
 'TS_2',
 'T2MDEW_2',
 'T2MWET_2',
 'T2M_MAX_2',
 'T2M_MIN_2',
 'T2M_2',
 'QV2M_2',
 'RH2M_2',
 'PRECTOTCORR_2',
 'PS_2',
 'WS2M_2',
 'WS2M_MAX_2',
 'WS2M_MIN_2',
 'WS2M_RANGE_2',
 'WS10M_2',
 'WS10M_MAX_2',
 'WS10M_MIN_2',
 'WS10M_RANGE_2',
 'WS50M_2',
 'WS50M_MAX_2',
 'WS50M_MIN_2',
 'WS50M_RANGE_2',
 'GWETTOP_2',
 'GWETROOT_2',
 'GWETPROF_2',
 'ALLSKY_SFC_SW_DWN_2',
 'CLRSKY_SFC_SW_DWN_2',
 'TOA_SW_DWN_

#### Iremos criar um dataset menor para cada mês, mudar o nome dos indices para o indice_mes e depois juntar todos em um dataset com todos os indices ordenados por mês.

In [50]:
l_m=[12,1,2,3,4]

df_m=datanew[col].loc[datanew['month']==11].rename(columns=dict(zip(col[0:34],lista_indices[1:35]))).drop(columns='month')

a=35
b=69
for i in l_m:
        
    df_m2=datanew[col].loc[datanew['month']==i].rename(columns=dict(zip(col[0:34],lista_indices[a:b]))).drop(columns='month')
    a=b
    b=b+34
    df_m=pd.concat([df_m.reset_index(drop=True),df_m2.reset_index(drop=True)], axis=1)
    

In [51]:
df_m

,T2M_RANGE_1,TS_1,T2MDEW_1,T2MWET_1,T2M_MAX_1,T2M_MIN_1,T2M_1,QV2M_1,RH2M_1,PRECTOTCORR_1,...,WS50M_6,WS50M_MAX_6,WS50M_MIN_6,WS50M_RANGE_6,GWETTOP_6,GWETROOT_6,GWETPROF_6,ALLSKY_SFC_SW_DWN_6,CLRSKY_SFC_SW_DWN_6,TOA_SW_DWN_6
0,11.511667,24.195667,18.900333,21.303667,29.609000,18.098000,23.705667,14.383333,76.684000,5.791667,...,0.598000,0.600667,16.357333,20.808667,29.943000,88.034000,111.137667,11.629667,0.323000,-31.676667
1,12.229000,25.032667,18.453000,21.416667,30.776000,18.545333,24.383333,13.975667,72.224667,2.867667,...,0.601000,0.603667,17.405333,20.792333,29.980333,93.299000,110.707333,12.203000,0.338333,-64.969333
2,12.259000,26.856000,18.589000,22.329333,32.465333,20.209333,26.068000,14.196667,66.865000,3.950000,...,0.688667,0.700333,17.724667,21.210667,30.022333,94.213333,112.331333,12.313000,0.340667,1.763000
3,12.544000,25.461667,17.990667,21.352333,31.216000,18.674333,24.711667,13.667333,69.588333,6.594667,...,0.712333,0.724667,18.241000,21.113667,30.065333,97.365333,112.391667,12.708667,0.356333,-64.868333
4,11.945667,25.957667,18.322000,21.687667,31.262333,19.315667,25.053000,13.869333,69.645667,4.463333,...,0.666333,0.675000,16.684000,20.802667,29.927000,89.587000,110.953000,11.790667,0.326333,-31.669333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,12.171333,23.178000,17.295333,20.015000,29.147667,16.975333,22.734667,13.314333,74.488333,2.473000,...,0.714667,0.709000,18.254333,20.650000,29.105667,97.099000,109.976000,12.546333,0.340333,-31.575333
336,11.504333,22.031333,17.722667,19.806667,27.956000,16.451667,21.888000,13.708333,79.546667,6.257000,...,0.748667,0.738000,15.479667,20.404667,29.150333,81.761333,107.510333,10.698333,0.288000,-65.201333
337,10.868333,23.022667,18.585000,20.717000,28.422000,17.553333,22.847667,14.406667,78.705000,4.125667,...,0.780667,0.772667,19.519667,20.833667,29.195000,102.869000,110.192333,13.168333,0.340667,-64.973333
338,13.145333,25.082333,17.663667,21.055000,31.414667,18.268333,24.443667,13.629000,69.940000,8.222000,...,0.699000,0.694333,15.233667,20.473000,29.239667,81.243000,108.566667,10.692000,0.291333,-31.829000


#### Reinserindo ano e codigo do ibge:

In [52]:
df_m['codigo_ibge']=datanew['codigo_ibge'].loc[datanew['month']==1].reset_index(drop=True)
df_m['year']=datanew['year'].loc[datanew['month']==1].reset_index(drop=True)

In [53]:
df_m.head()

,T2M_RANGE_1,TS_1,T2MDEW_1,T2MWET_1,T2M_MAX_1,T2M_MIN_1,T2M_1,QV2M_1,RH2M_1,PRECTOTCORR_1,...,WS50M_MIN_6,WS50M_RANGE_6,GWETTOP_6,GWETROOT_6,GWETPROF_6,ALLSKY_SFC_SW_DWN_6,CLRSKY_SFC_SW_DWN_6,TOA_SW_DWN_6,codigo_ibge,year
0,11.511667,24.195667,18.900333,21.303667,29.609000,18.098000,23.705667,14.383333,76.684000,5.791667,...,16.357333,20.808667,29.943000,88.034000,111.137667,11.629667,0.323000,-31.676667,4128005,2004
1,12.229000,25.032667,18.453000,21.416667,30.776000,18.545333,24.383333,13.975667,72.224667,2.867667,...,17.405333,20.792333,29.980333,93.299000,110.707333,12.203000,0.338333,-64.969333,4128005,2005
2,12.259000,26.856000,18.589000,22.329333,32.465333,20.209333,26.068000,14.196667,66.865000,3.950000,...,17.724667,21.210667,30.022333,94.213333,112.331333,12.313000,0.340667,1.763000,4128005,2006
3,12.544000,25.461667,17.990667,21.352333,31.216000,18.674333,24.711667,13.667333,69.588333,6.594667,...,18.241000,21.113667,30.065333,97.365333,112.391667,12.708667,0.356333,-64.868333,4128005,2007
4,11.945667,25.957667,18.322000,21.687667,31.262333,19.315667,25.053000,13.869333,69.645667,4.463333,...,16.684000,20.802667,29.927000,89.587000,110.953000,11.790667,0.326333,-31.669333,4128005,2008


In [54]:
#X_train=df_m.loc[df_m['year']<2018]
#X_test=df_m.loc[df_m['year']>2017]

X_train=df_m.loc[df_m['year']<2017]
X_test=df_m.loc[df_m['year']==2017]
Y_train_1=Y_train.drop(columns=['2017'])

In [55]:
Y_train_1

,codigo_ibge,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,4100103,2600.0,1750.0,2100.0,2600.0,2600.0,2080.0,2590.0,3350.0,2320.0,3100.0,1460.0,2640.0,2760.0
1,4100459,3100.0,2460.0,2950.0,2950.0,2749.0,2749.0,3250.0,3200.0,2197.0,3223.0,3471.0,3099.0,2871.0
2,4100608,1487.0,1365.0,1734.0,2727.0,2400.0,1620.0,2899.0,2899.0,1737.0,2880.0,2000.0,1800.0,2221.0
3,4100707,2300.0,2300.0,2700.0,3193.0,2701.0,2097.0,3130.0,3310.0,2000.0,3000.0,3192.0,3488.0,2589.0
4,4100509,1050.0,1800.0,1255.0,2355.0,2600.0,1300.0,3000.0,2773.0,815.0,2137.0,1800.0,2455.0,2780.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,4128559,2300.0,2628.0,2229.0,3000.0,3470.0,1999.0,3385.0,3498.0,1611.0,3743.0,3216.0,3394.0,3500.0
345,4128609,2375.0,1540.0,2167.0,2248.0,2278.0,1892.0,2625.0,3166.0,1941.0,2714.0,2582.0,3111.0,2690.0
346,4128658,2400.0,1900.0,3100.0,3075.0,3113.0,2500.0,3200.0,3350.0,2236.0,2748.0,3048.0,3049.0,1360.0
347,4128708,2477.0,1823.0,2948.0,2835.0,3052.0,3156.0,2971.0,3526.0,1854.0,3403.0,3309.0,3212.0,2836.0


#### Scalando os dados:

In [56]:
colunas_=list(X_train.keys())

colunas_.remove('year')
colunas_.remove('codigo_ibge')
#removendo os dados que não serão escalados

In [57]:
len(colunas_)

204

In [58]:
X_train[colunas_]

,T2M_RANGE_1,TS_1,T2MDEW_1,T2MWET_1,T2M_MAX_1,T2M_MIN_1,T2M_1,QV2M_1,RH2M_1,PRECTOTCORR_1,...,WS50M_6,WS50M_MAX_6,WS50M_MIN_6,WS50M_RANGE_6,GWETTOP_6,GWETROOT_6,GWETPROF_6,ALLSKY_SFC_SW_DWN_6,CLRSKY_SFC_SW_DWN_6,TOA_SW_DWN_6
0,11.511667,24.195667,18.900333,21.303667,29.609000,18.098000,23.705667,14.383333,76.684000,5.791667,...,0.598000,0.600667,16.357333,20.808667,29.943000,88.034000,111.137667,11.629667,0.323000,-31.676667
1,12.229000,25.032667,18.453000,21.416667,30.776000,18.545333,24.383333,13.975667,72.224667,2.867667,...,0.601000,0.603667,17.405333,20.792333,29.980333,93.299000,110.707333,12.203000,0.338333,-64.969333
2,12.259000,26.856000,18.589000,22.329333,32.465333,20.209333,26.068000,14.196667,66.865000,3.950000,...,0.688667,0.700333,17.724667,21.210667,30.022333,94.213333,112.331333,12.313000,0.340667,1.763000
3,12.544000,25.461667,17.990667,21.352333,31.216000,18.674333,24.711667,13.667333,69.588333,6.594667,...,0.712333,0.724667,18.241000,21.113667,30.065333,97.365333,112.391667,12.708667,0.356333,-64.868333
4,11.945667,25.957667,18.322000,21.687667,31.262333,19.315667,25.053000,13.869333,69.645667,4.463333,...,0.666333,0.675000,16.684000,20.802667,29.927000,89.587000,110.953000,11.790667,0.326333,-31.669333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,14.426667,25.261667,16.139000,20.237667,32.059333,17.634667,24.337000,12.395667,64.638000,2.161000,...,0.671000,0.664333,14.861000,20.668667,29.120333,78.810667,109.196333,10.379333,0.278333,-31.903333
332,12.450000,23.213667,17.415667,20.091333,29.323667,16.874000,22.768667,13.376667,74.499667,3.350000,...,0.719667,0.713000,17.481333,20.853667,29.162333,91.607333,109.504333,11.840667,0.310667,-98.466667
333,12.250667,24.100333,17.243000,20.364667,29.994667,17.744333,23.486333,13.432667,72.207333,3.747667,...,0.786667,0.773667,15.316667,20.408333,29.213000,81.540333,108.373333,10.639000,0.285333,1.492333
334,8.834000,23.280333,20.381000,21.796000,27.811667,18.978000,23.209667,16.050333,85.300000,11.334000,...,0.678333,0.674000,17.256000,20.714667,29.268000,91.179333,109.415000,11.846333,0.310667,-31.752667


### Podemos criar um pipeline com as etapas do processo para testes:

In [59]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV

In [61]:
seed=2
modelos_reg=[
    Ridge(random_state=seed, tol=10 ),
    Lasso(random_state=seed, tol=1),
    SVR(kernel='linear', C=1e2, degree=5),
    RandomForestRegressor(random_state=seed),
    DecisionTreeRegressor(),
    KNeighborsRegressor(),
    AdaBoostRegressor(),
    GradientBoostingRegressor(),    
    SVC(kernel='rbf',gamma='scale', C=1),
    SVC(kernel="rbf",C=0.025,probability=True),
    
    #learning_rate= 0.04, max_depth= 4, n_estimators=500, subsample= 0.9),
    #RandomForestRegressor(n_estimators=100, random_state=0)
    ]

#pipe.fit(X_train,Y_train)

In [63]:
X_train.drop(columns=['codigo_ibge'])

,T2M_RANGE_1,TS_1,T2MDEW_1,T2MWET_1,T2M_MAX_1,T2M_MIN_1,T2M_1,QV2M_1,RH2M_1,PRECTOTCORR_1,...,WS50M_MAX_6,WS50M_MIN_6,WS50M_RANGE_6,GWETTOP_6,GWETROOT_6,GWETPROF_6,ALLSKY_SFC_SW_DWN_6,CLRSKY_SFC_SW_DWN_6,TOA_SW_DWN_6,year
0,11.511667,24.195667,18.900333,21.303667,29.609000,18.098000,23.705667,14.383333,76.684000,5.791667,...,0.600667,16.357333,20.808667,29.943000,88.034000,111.137667,11.629667,0.323000,-31.676667,2004
1,12.229000,25.032667,18.453000,21.416667,30.776000,18.545333,24.383333,13.975667,72.224667,2.867667,...,0.603667,17.405333,20.792333,29.980333,93.299000,110.707333,12.203000,0.338333,-64.969333,2005
2,12.259000,26.856000,18.589000,22.329333,32.465333,20.209333,26.068000,14.196667,66.865000,3.950000,...,0.700333,17.724667,21.210667,30.022333,94.213333,112.331333,12.313000,0.340667,1.763000,2006
3,12.544000,25.461667,17.990667,21.352333,31.216000,18.674333,24.711667,13.667333,69.588333,6.594667,...,0.724667,18.241000,21.113667,30.065333,97.365333,112.391667,12.708667,0.356333,-64.868333,2007
4,11.945667,25.957667,18.322000,21.687667,31.262333,19.315667,25.053000,13.869333,69.645667,4.463333,...,0.675000,16.684000,20.802667,29.927000,89.587000,110.953000,11.790667,0.326333,-31.669333,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,14.426667,25.261667,16.139000,20.237667,32.059333,17.634667,24.337000,12.395667,64.638000,2.161000,...,0.664333,14.861000,20.668667,29.120333,78.810667,109.196333,10.379333,0.278333,-31.903333,2012
332,12.450000,23.213667,17.415667,20.091333,29.323667,16.874000,22.768667,13.376667,74.499667,3.350000,...,0.713000,17.481333,20.853667,29.162333,91.607333,109.504333,11.840667,0.310667,-98.466667,2013
333,12.250667,24.100333,17.243000,20.364667,29.994667,17.744333,23.486333,13.432667,72.207333,3.747667,...,0.773667,15.316667,20.408333,29.213000,81.540333,108.373333,10.639000,0.285333,1.492333,2014
334,8.834000,23.280333,20.381000,21.796000,27.811667,18.978000,23.209667,16.050333,85.300000,11.334000,...,0.674000,17.256000,20.714667,29.268000,91.179333,109.415000,11.846333,0.310667,-31.752667,2015


In [62]:
for modelo in modelos_reg:
    
    pipe= Pipeline([('scaler', StandardScaler()),
                   ('model',modelo)])


    df_error=pd.DataFrame(columns=['codigo_ibge','2017_prev','2017_true'])


    X_train_menor=X_train.drop(columns=['codigo_ibge'])
    Y_train_menor=Y_train_1[Y_train_1['codigo_ibge']==i].drop(columns='codigo_ibge').T

    pipe.fit(X_train_menor, Y_train_menor.values.ravel())


    X_test_menor=X_test.loc[X_test['codigo_ibge']==i].drop(columns=['codigo_ibge'])


        #pipe.fit_predict()
    predicoes=pipe.predict(X_test_menor)


    data_error={'codigo_ibge':i,'2017_prev':float(predicoes),'2017_true':float(Y_train[Y_train['codigo_ibge']==i]['2017'])}


    data_error=pd.DataFrame([data_error])

    df_error=pd.concat([df_error,data_error])

    df_error=df_error.reset_index(drop=True)

    print(modelo)
    print(mean_absolute_error(list(df_error['2017_prev']), list(df_error['2017_true'])))

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=2, solver='auto', tol=10)
801.6267649652216
Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=2,
      selection='cyclic', tol=1, warm_start=False)
566.7480540870507
SVR(C=100.0, cache_size=200, coef0=0.0, degree=5, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
801.1150117578538
RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=2, verbose=0, 

In [65]:
df_error=df_error.reset_index(drop=True)

In [66]:
df_error

,codigo_ibge,2017_prev,2017_true
0,4102000,3245.227026,3967.0
1,4104303,3300.860056,3840.0
2,4104428,3209.501368,4070.0
3,4104808,3547.007947,3900.0
4,4104907,3523.977032,4000.0
5,4109401,3478.437489,4120.0
6,4113205,3191.393096,3350.0
7,4113700,2928.542771,3250.0
8,4113734,2763.609436,3471.0
9,4114005,3285.584331,3350.0


In [35]:
mean_absolute_error(list(df_error['2017_prev']), list(df_error['2017_true']))

697.0195